In [48]:
import pandas as pd
from pandas.api.types import CategoricalDtype 

#### read tsv file with **sep='\t'**

In [10]:
data = pd.read_csv("train.tsv", sep='\t')

In [11]:
pd.set_option('display.max_columns', None)

In [12]:
data.head()

,id,Y,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0,p,f,y,n,f,f,f,c,n,b,t,?,s,k,w,w,p,w,o,e,w,v,d
1,2,p,f,y,y,f,f,f,c,b,p,e,b,k,k,b,p,p,w,o,l,h,y,g
2,3,e,b,y,w,t,l,f,c,b,k,e,c,s,s,w,w,p,w,o,p,k,n,m
3,5,p,x,s,b,t,f,f,c,b,h,t,b,s,s,w,w,p,w,o,p,h,v,u
4,7,p,x,s,w,t,f,f,c,b,w,t,b,s,f,w,w,p,w,o,p,h,v,g


#### check null

In [22]:
data.isnull().sum()

id                          0
Y                           0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [27]:
y = data.Y
y = y.to_frame()

#### set p = 1 and e = 0 
#### while 1 means toxic and 0 means safe

In [30]:
toxic = y.Y == 'p'
column_name = 'Y'
y.loc[toxic, column_name] = 1
safe = y.Y == 'e'
column_name = 'Y'
y.loc[safe, column_name] = 0

In [31]:
y

,Y
0,1
1,1
2,0
3,1
4,1
...,...
4057,1
4058,1
4059,1
4060,1


In [38]:
y['Y'].value_counts()

0    2103
1    1959
Name: Y, dtype: int64

In [39]:
features = ['cap-shape', 'cap-color', 'bruises', 'odor', 'habitat']

In [40]:
X = data[features]

In [41]:
X

,cap-shape,cap-color,bruises,odor,habitat
0,f,n,f,f,d
1,f,y,f,f,g
2,b,w,t,l,m
3,x,b,t,f,u
4,x,w,t,f,g
...,...,...,...,...,...
4057,f,n,f,s,p
4058,x,y,f,f,p
4059,k,n,f,y,d
4060,x,n,t,p,u


In [43]:
type(X.columns)

pandas.core.indexes.base.Index

In [46]:
for feature in X.columns:
    print(X[feature].value_counts())

x    1809
f    1573
k     423
b     244
s      10
c       3
Name: cap-shape, dtype: int64
n    1166
g     876
e     743
w     541
y     527
b      93
p      80
c      18
r      10
u       8
Name: cap-color, dtype: int64
f    2367
t    1695
Name: bruises, dtype: int64
n    1747
f    1078
s     291
y     287
l     210
a     201
p     130
c      99
m      19
Name: odor, dtype: int64
d    1562
g    1059
p     581
l     407
u     192
m     153
w     108
Name: habitat, dtype: int64
